In [2]:
# Testing some hypothesis

In [3]:
import tensorflow as tf

rel_points = tf.convert_to_tensor(
    [[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
print(rel_points)

tf.Tensor(
[[ 1  0  0]
 [ 0  1  0]
 [ 0  0  1]
 [-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]], shape=(6, 3), dtype=int32)


2024-02-04 17:53:46.727893: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-02-04 17:53:46.727913: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-02-04 17:53:46.727917: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-02-04 17:53:46.727946: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-04 17:53:46.727961: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
centers = tf.convert_to_tensor([[1, 1, 1, 1], [2, 2, 2, 1], [3, 3, 3, 1], [10, 10, 10, 1]])
print(centers)

tf.Tensor(
[[ 1  1  1  1]
 [ 2  2  2  1]
 [ 3  3  3  1]
 [10 10 10  1]], shape=(4, 4), dtype=int32)


In [5]:
coords, feats = tf.split(centers, [3, 1], axis=-1)
print(coords)
print(feats)

tf.Tensor(
[[ 1  1  1]
 [ 2  2  2]
 [ 3  3  3]
 [10 10 10]], shape=(4, 3), dtype=int32)
tf.Tensor(
[[1]
 [1]
 [1]
 [1]], shape=(4, 1), dtype=int32)


In [6]:
n, d = coords.shape
print(f"n={n}, d={d}")

n=4, d=3


In [7]:
m, _ = rel_points.shape
print(f"num centers = {m}")

num centers = 6


In [8]:
coords_broadcasted = tf.broadcast_to(tf.reshape(coords, (n, 1, d)), shape=(n, m, d))
print(coords_broadcasted)

tf.Tensor(
[[[ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]]

 [[ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]]

 [[ 3  3  3]
  [ 3  3  3]
  [ 3  3  3]
  [ 3  3  3]
  [ 3  3  3]
  [ 3  3  3]]

 [[10 10 10]
  [10 10 10]
  [10 10 10]
  [10 10 10]
  [10 10 10]
  [10 10 10]]], shape=(4, 6, 3), dtype=int32)


In [9]:
center_broadcasted = tf.broadcast_to(tf.reshape(rel_points, (1, m, d)), shape=(n, m, d))
print(center_broadcasted)

tf.Tensor(
[[[ 1  0  0]
  [ 0  1  0]
  [ 0  0  1]
  [-1  0  0]
  [ 0 -1  0]
  [ 0  0 -1]]

 [[ 1  0  0]
  [ 0  1  0]
  [ 0  0  1]
  [-1  0  0]
  [ 0 -1  0]
  [ 0  0 -1]]

 [[ 1  0  0]
  [ 0  1  0]
  [ 0  0  1]
  [-1  0  0]
  [ 0 -1  0]
  [ 0  0 -1]]

 [[ 1  0  0]
  [ 0  1  0]
  [ 0  0  1]
  [-1  0  0]
  [ 0 -1  0]
  [ 0  0 -1]]], shape=(4, 6, 3), dtype=int32)


In [10]:
computed_points = center_broadcasted + coords_broadcasted
print(computed_points)

tf.Tensor(
[[[ 2  1  1]
  [ 1  2  1]
  [ 1  1  2]
  [ 0  1  1]
  [ 1  0  1]
  [ 1  1  0]]

 [[ 3  2  2]
  [ 2  3  2]
  [ 2  2  3]
  [ 1  2  2]
  [ 2  1  2]
  [ 2  2  1]]

 [[ 4  3  3]
  [ 3  4  3]
  [ 3  3  4]
  [ 2  3  3]
  [ 3  2  3]
  [ 3  3  2]]

 [[11 10 10]
  [10 11 10]
  [10 10 11]
  [ 9 10 10]
  [10  9 10]
  [10 10  9]]], shape=(4, 6, 3), dtype=int32)


In [11]:
# Now let's get some points to convolute

inputs = tf.convert_to_tensor([[2, 1, 2], [3, 4, 5], [6, 7, 8], [3, 2, 1], [5, 4, 3], [6, 6, 6]])
print(inputs)

tf.Tensor(
[[2 1 2]
 [3 4 5]
 [6 7 8]
 [3 2 1]
 [5 4 3]
 [6 6 6]], shape=(6, 3), dtype=int32)


In [12]:
nn, _ = inputs.shape
print(f"Num points = {nn}")

Num points = 6


In [13]:
# Compute relative distances from focus points

inputs_b = tf.broadcast_to(tf.reshape(inputs, (nn, 1, 1, 3)), shape=(nn, n, m, d))
print(inputs_b)

tf.Tensor(
[[[[2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]]

  [[2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]]

  [[2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]]

  [[2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]
   [2 1 2]]]


 [[[3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]]

  [[3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]]

  [[3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]]

  [[3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]
   [3 4 5]]]


 [[[6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]]

  [[6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]]

  [[6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]]

  [[6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]
   [6 7 8]]]


 [[[3 2 1]
   [3 2 1]
   [3 2 1]
   [3 2 1]
   [3 2 1]
   [3 2 1]]

  [[3 2 1]
   [3 2 1]
   [3 2 1]
   [3 2 1]
   [3 2 1]
   [3 2 1]]

  [[3 2 1]
   [3 2 1]
   [3 2 1

In [14]:
computed_points_b = tf.broadcast_to(tf.reshape(computed_points, (1, n, m, d)), (nn, n, m, d))
print(computed_points_b)

tf.Tensor(
[[[[ 2  1  1]
   [ 1  2  1]
   [ 1  1  2]
   [ 0  1  1]
   [ 1  0  1]
   [ 1  1  0]]

  [[ 3  2  2]
   [ 2  3  2]
   [ 2  2  3]
   [ 1  2  2]
   [ 2  1  2]
   [ 2  2  1]]

  [[ 4  3  3]
   [ 3  4  3]
   [ 3  3  4]
   [ 2  3  3]
   [ 3  2  3]
   [ 3  3  2]]

  [[11 10 10]
   [10 11 10]
   [10 10 11]
   [ 9 10 10]
   [10  9 10]
   [10 10  9]]]


 [[[ 2  1  1]
   [ 1  2  1]
   [ 1  1  2]
   [ 0  1  1]
   [ 1  0  1]
   [ 1  1  0]]

  [[ 3  2  2]
   [ 2  3  2]
   [ 2  2  3]
   [ 1  2  2]
   [ 2  1  2]
   [ 2  2  1]]

  [[ 4  3  3]
   [ 3  4  3]
   [ 3  3  4]
   [ 2  3  3]
   [ 3  2  3]
   [ 3  3  2]]

  [[11 10 10]
   [10 11 10]
   [10 10 11]
   [ 9 10 10]
   [10  9 10]
   [10 10  9]]]


 [[[ 2  1  1]
   [ 1  2  1]
   [ 1  1  2]
   [ 0  1  1]
   [ 1  0  1]
   [ 1  1  0]]

  [[ 3  2  2]
   [ 2  3  2]
   [ 2  2  3]
   [ 1  2  2]
   [ 2  1  2]
   [ 2  2  1]]

  [[ 4  3  3]
   [ 3  4  3]
   [ 3  3  4]
   [ 2  3  3]
   [ 3  2  3]
   [ 3  3  2]]

  [[11 10 10]
   [10 11 10]
   [10 10 1

In [15]:
deltas = inputs_b - computed_points_b
print(deltas)

tf.Tensor(
[[[[  0   0   1]
   [  1  -1   1]
   [  1   0   0]
   [  2   0   1]
   [  1   1   1]
   [  1   0   2]]

  [[ -1  -1   0]
   [  0  -2   0]
   [  0  -1  -1]
   [  1  -1   0]
   [  0   0   0]
   [  0  -1   1]]

  [[ -2  -2  -1]
   [ -1  -3  -1]
   [ -1  -2  -2]
   [  0  -2  -1]
   [ -1  -1  -1]
   [ -1  -2   0]]

  [[ -9  -9  -8]
   [ -8 -10  -8]
   [ -8  -9  -9]
   [ -7  -9  -8]
   [ -8  -8  -8]
   [ -8  -9  -7]]]


 [[[  1   3   4]
   [  2   2   4]
   [  2   3   3]
   [  3   3   4]
   [  2   4   4]
   [  2   3   5]]

  [[  0   2   3]
   [  1   1   3]
   [  1   2   2]
   [  2   2   3]
   [  1   3   3]
   [  1   2   4]]

  [[ -1   1   2]
   [  0   0   2]
   [  0   1   1]
   [  1   1   2]
   [  0   2   2]
   [  0   1   3]]

  [[ -8  -6  -5]
   [ -7  -7  -5]
   [ -7  -6  -6]
   [ -6  -6  -5]
   [ -7  -5  -5]
   [ -7  -6  -4]]]


 [[[  4   6   7]
   [  5   5   7]
   [  5   6   6]
   [  6   6   7]
   [  5   7   7]
   [  5   6   8]]

  [[  3   5   6]
   [  4   4   6]
   [  4   5   5

In [16]:
# Now dot prod on coords to get the modulus

norm2 = tf.reduce_sum(deltas * deltas, axis=3)
print(norm2)

tf.Tensor(
[[[  1   3   1   5   3   5]
  [  2   4   2   2   0   2]
  [  9  11   9   5   3   5]
  [226 228 226 194 192 194]]

 [[ 26  24  22  34  36  38]
  [ 13  11   9  17  19  21]
  [  6   4   2   6   8  10]
  [125 123 121  97  99 101]]

 [[101  99  97 121 123 125]
  [ 70  68  66  86  88  90]
  [ 45  43  41  57  59  61]
  [ 38  36  34  22  24  26]]

 [[  2   4   6  10   8   6]
  [  1   3   5   5   3   1]
  [  6   8  10   6   4   2]
  [209 211 213 181 179 177]]

 [[ 22  24  26  38  36  34]
  [  9  11  13  21  19  17]
  [  2   4   6  10   8   6]
  [121 123 125 101  99  97]]

 [[ 66  66  66  86  86  86]
  [ 41  41  41  57  57  57]
  [ 22  22  22  34  34  34]
  [ 57  57  57  41  41  41]]], shape=(6, 4, 6), dtype=int32)


In [17]:
inv_r = 1 / (tf.cast(norm2, tf.float32) + 0.01)
print(inv_r)

tf.Tensor(
[[[9.9009901e-01 3.3222592e-01 9.9009901e-01 1.9960079e-01 3.3222592e-01
   1.9960079e-01]
  [4.9751243e-01 2.4937655e-01 4.9751243e-01 4.9751243e-01 1.0000000e+02
   4.9751243e-01]
  [1.1098779e-01 9.0826519e-02 1.1098779e-01 1.9960079e-01 3.3222592e-01
   1.9960079e-01]
  [4.4245832e-03 4.3857726e-03 4.4245832e-03 5.1543736e-03 5.2080620e-03
   5.1543736e-03]]

 [[3.8446750e-02 4.1649312e-02 4.5433894e-02 2.9403118e-02 2.7770065e-02
   2.6308868e-02]
  [7.6863952e-02 9.0826519e-02 1.1098779e-01 5.8788948e-02 5.2603893e-02
   4.7596384e-02]
  [1.6638935e-01 2.4937655e-01 4.9751243e-01 1.6638935e-01 1.2484394e-01
   9.9900097e-02]
  [7.9993596e-03 8.1294207e-03 8.2637798e-03 1.0308215e-02 1.0099989e-02
   9.9000093e-03]]

 [[9.9000093e-03 1.0099989e-02 1.0308215e-02 8.2637798e-03 8.1294207e-03
   7.9993596e-03]
  [1.4283673e-02 1.4703720e-02 1.5149219e-02 1.1626555e-02 1.1362345e-02
   1.1109876e-02]
  [2.2217287e-02 2.3250408e-02 2.4384297e-02 1.7540783e-02 1.6946280e-02
  

In [18]:
x1 = tf.reduce_mean(inv_r, axis=(0, 1))
print(x1)

tf.Tensor([0.18287186 0.09776235 0.1336358  0.08282074 4.248891   0.13195896], shape=(6,), dtype=float32)


In [19]:
d = 3
tf.concat([tf.one_hot(i, d) * tf.convert_to_tensor([[-1.], [1.]]) for i in range(d)], axis=0)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[-1., -0., -0.],
       [ 1.,  0.,  0.],
       [-0., -1., -0.],
       [ 0.,  1.,  0.],
       [-0., -0., -1.],
       [ 0.,  0.,  1.]], dtype=float32)>

In [31]:
a = tf.convert_to_tensor([[1, 2, 3]], dtype=tf.float64)
print(a)
b = tf.convert_to_tensor([[0], [1]], dtype=tf.float64)
print(b)

tf.Tensor([[1. 2. 3.]], shape=(1, 3), dtype=float64)
tf.Tensor(
[[0.]
 [1.]], shape=(2, 1), dtype=float64)


In [32]:
c = a + b
c

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[1., 2., 3.],
       [2., 3., 4.]])>

In [35]:
tf.norm(c, axis=1)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([3.74165739, 5.38516481])>

In [49]:
import math

angles = tf.convert_to_tensor(
    [[j * 0.125 - 0.125, i * 0.125 + (j - 1) * 0.0625] for j in range(0, 3) for i in
     range(0, 8)] + [[-0.25, 0], [0.25, 0]]) * math.pi * 2
angles

<tf.Tensor: shape=(26, 2), dtype=float32, numpy=
array([[-0.7853982, -0.3926991],
       [-0.7853982,  0.3926991],
       [-0.7853982,  1.1780972],
       [-0.7853982,  1.9634955],
       [-0.7853982,  2.7488937],
       [-0.7853982,  3.5342917],
       [-0.7853982,  4.31969  ],
       [-0.7853982,  5.105088 ],
       [ 0.       ,  0.       ],
       [ 0.       ,  0.7853982],
       [ 0.       ,  1.5707964],
       [ 0.       ,  2.3561945],
       [ 0.       ,  3.1415927],
       [ 0.       ,  3.926991 ],
       [ 0.       ,  4.712389 ],
       [ 0.       ,  5.4977875],
       [ 0.7853982,  0.3926991],
       [ 0.7853982,  1.1780972],
       [ 0.7853982,  1.9634955],
       [ 0.7853982,  2.7488937],
       [ 0.7853982,  3.5342917],
       [ 0.7853982,  4.31969  ],
       [ 0.7853982,  5.105088 ],
       [ 0.7853982,  5.8904862],
       [-1.5707964,  0.       ],
       [ 1.5707964,  0.       ]], dtype=float32)>

In [50]:
points = tf.convert_to_tensor(
    [[tf.sin(a), tf.cos(a) * tf.cos(b), tf.cos(a) * tf.sin(b)] for a, b in angles])
points

<tf.Tensor: shape=(26, 3), dtype=float32, numpy=
array([[-7.0710683e-01,  6.5328145e-01, -2.7059805e-01],
       [-7.0710683e-01,  6.5328145e-01,  2.7059805e-01],
       [-7.0710683e-01,  2.7059805e-01,  6.5328145e-01],
       [-7.0710683e-01, -2.7059811e-01,  6.5328145e-01],
       [-7.0710683e-01, -6.5328151e-01,  2.7059793e-01],
       [-7.0710683e-01, -6.5328145e-01, -2.7059805e-01],
       [-7.0710683e-01, -2.7059782e-01, -6.5328157e-01],
       [-7.0710683e-01,  2.7059817e-01, -6.5328139e-01],
       [ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  7.0710677e-01,  7.0710683e-01],
       [ 0.0000000e+00, -4.3711388e-08,  1.0000000e+00],
       [ 0.0000000e+00, -7.0710677e-01,  7.0710677e-01],
       [ 0.0000000e+00, -1.0000000e+00, -8.7422777e-08],
       [ 0.0000000e+00, -7.0710671e-01, -7.0710689e-01],
       [ 0.0000000e+00,  1.1924881e-08, -1.0000000e+00],
       [ 0.0000000e+00,  7.0710701e-01, -7.0710653e-01],
       [ 7.0710683e-01,  6.5328145e-01,

In [69]:
 tf.keras.Sequential([layer for layers in [[
    tf.keras.layers.Dense(
        units=2),
    tf.keras.layers.Dropout(
        rate=0.10),
    tf.keras.layers.LeakyReLU()
] for _ in range(3)] for layer in layers])

In [68]:
[layer for layers in [[
    tf.keras.layers.Dense(
        units=2),
    tf.keras.layers.Dropout(
        rate=0.10),
    tf.keras.layers.LeakyReLU()
] for _ in range(3)] for layer in layers]